In [ ]:
import sys
import os

# Addition of the scripts file Ajouter le dossier `scripts/` au chemin d'accès Python
sys.path.append(os.path.abspath("scripts"))

# Import custom functions
from scripts.data_processing import load_data, preprocess_data
from scripts.eda import exploratory_data_analysis
from scripts.modelling import train_models
from scripts.evaluation import evaluate_models, predict_probabilities, robustness_check

# Load data
file_path = "/Users/camillelemaignen/Project_SIA_Driving_Test/data/DataDSVA_Python.xlsm"
df = load_data(file_path)

#Data cleaning and preprocessing
df_filtered = preprocess_data(df)

In [ ]:
# Exploratory Data Analysis
exploratory_data_analysis(df)

In [ ]:
# Model training
logit_model, probit_model = train_models(df_filtered)

In [ ]:
# Model evaluation and robustness
evaluate_models(logit_model, probit_model, df_filtered)
robustness_check(df_filtered, logit_model)

In [ ]:
# Make predictions
predict_probabilities(logit_model)